In [ ]:
!nvidia-smi

Sun Sep  3 14:51:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Second Method

In [ ]:
# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

In [ ]:
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
!pip install -qq langchain wget
!pip install gguf  #https://github.com/ggerganov/llama.cpp/tree/master/gguf-py
!git clone https://github.com/ggerganov/llama.cpp
!pip -qq install git+https://github.com/huggingface/transformers
!pip install --upgrade numba tensorflow
!pip install numpy==1.18
#Assuming you are using a GPU
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip -qq install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

In [ ]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

from huggingface_hub import hf_hub_download
repo_id="TheBloke/Llama-2-13B-GGML"; filename="llama-2-13b.ggmlv3.q5_1.bin"

In [ ]:
!intel-smi


In [ ]:
hf_hub_download(
    repo_id=repo_id, filename=filename,
    local_dir="/content"
)

'/content/llama-2-13b.ggmlv3.q5_1.bin'

In [ ]:
!python /content/llama.cpp/convert-llama-ggmlv3-to-gguf.py --input `ls -t /content/*ggmlv3*.bin | head -1` --output `ls -t /content/*ggmlv3*.bin | head -1`.gguf

filename=filename+".gguf"

* Using config: Namespace(input=PosixPath('/content/llama-2-13b.ggmlv3.q5_1.bin'), output=PosixPath('/content/llama-2-13b.ggmlv3.q5_1.bin.gguf'), name=None, desc=None, gqa=1, eps='5.0e-06', context_length=2048, model_metadata_dir=None, vocab_dir=None, vocabtype='spm')

=== WARNING === Be aware that this conversion script is best-effort. Use a native GGUF model if possible. === WARNING ===

* Scanning GGML input file
* GGML model hyperparameters: <Hyperparameters: n_vocab=32000, n_embd=5120, n_mult=256, n_head=40, n_layer=40, n_rot=128, n_ff=13824, ftype=9>

=== WARNING === Special tokens may not be converted correctly. Use --model-metadata-dir if possible === WARNING ===

* Preparing to save GGUF file
* Adding model parameters and KV items
* Adding 32000 vocab item(s)
* Adding 363 tensor(s)
    gguf: write header
    gguf: write metadata
    gguf: write tensors
* Successful completion. Output saved to: /content/llama-2-13b.ggmlv3.q5_1.bin.gguf


In [ ]:
n_gpu_layers = 62
n_batch = 512
n_threads=4
llm = LlamaCpp(
    model_path="/content/"+filename,
    n_threads=n_threads,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    n_ctx=2048,
    temperature=0.0,
    repeat_penalty=1.18,
    top_p=1,
    top_k=3,
    max_tokens=256,
    streaming=True,
    #verbose=True,
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [ ]:
# Test CPU times: user 4min 44s, sys: 43.5 ms, total: 4min 44s n_gpu_layers = 40 n_batch = 512
# Wall time: 2min 59s n_gpu_layers = 40 n_batch = 812

In [ ]:
%%time
conversation.predict(input="Q: What is the restoration forest?", stop=["Q:", "\n"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Q: What is the restoration forest?
AI:
 A: It's an area where trees are planted in order to restore forests that have been damaged or destroyed by human activity.
> Finished chain.
CPU times: user 4min 34s, sys: 4.71 ms, total: 4min 34s
Wall time: 3min 13s


" A: It's an area where trees are planted in order to restore forests that have been damaged or destroyed by human activity."

In [ ]:
print(memory.buffer)

Human: Q: What is the restoration forest?
AI:  A: It's an area where trees are planted in order to restore forests that have been damaged or destroyed by human activity.


#ConversationTokenBufferMemory

In [ ]:
from langchain.memory import ConversationTokenBufferMemory

ModuleNotFoundError: ignored

In [ ]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

memory.save_context({"input": "Q: What is the restoration forest?"},
                    {"output": "A: It's an area where trees are planted in order to restore forests that have been damaged or destroyed by human activity."})
memory.save_context({"input": "Q: Which plant or trees I should plant for make a succesfull restoration forest?"},
                    {"output": "A: There is no one-size-fits-all answer to this question, as the best plants and trees to use will depend on the specific conditions of your area and what you are trying to achieve. However, some general tips include choosing native species that are well adapted to local conditions, planting a variety of different types of trees to create a diverse ecosystem, and using trees that produce fruits or nuts for wildlife habitat."})

In [ ]:
conversation.predict(input="Hello", stop=["Q:", "\n"])



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello
AI:


Llama.generate: prefix-match hit


 Hi! I'm an artificial intelligence that can chat with you about anything. What would you like to talk about?
> Finished chain.


" Hi! I'm an artificial intelligence that can chat with you about anything. What would you like to talk about?"

In [ ]:
import pickle
pickled_str = pickle.dumps(conversation.memory)
conversation2 = ConversationChain(llm=llm, memory=pickle.loads(pickled_str))

In [ ]:
memory.save_context({"input": "Q: What is the restoration forest?"},
                    {"output": "A: It's an area where trees are planted in order to restore forests that have been damaged or destroyed by human activity."})
memory.save_context({"input": "Q: Which plant or trees I should plant for make a succesfull restoration forest?"},
                    {"output": "A: There is no one-size-fits-all answer to this question, as the best plants and trees to use will depend on the specific conditions of your area and what you are trying to achieve. However, some general tips include choosing native species that are well adapted to local conditions, planting a variety of different types of trees to create a diverse ecosystem, and using trees that produce fruits or nuts for wildlife habitat."})

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

{'history': ''}

#ConversationSummaryMemory

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

In [ ]:
# create a long string
schedule = "A: There is no one-size-fits-all answer to this question, as the best plants and trees to use will \
depend on the specific conditions of your area and what you are trying to achieve. \
However, some general tips include choosing native species that are well adapted to local conditions, \
planting a variety of different types of trees to create a diverse ecosystem, \
and using trees that produce fruits or nuts for wildlife habitat."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Q: What is the restoration forest?"},
                    {"output": "A: It's an area where trees are planted in order to restore forests that have been damaged or destroyed by human activity."})
memory.save_context({"input": "Q: Which plant or trees I should plant for make a succesfull restoration forest?"},
                    {"output": f"{schedule}"})

In [ ]:
memory.load_memory_variables({})

In [ ]:
# Set the model
n_gpu_layers = 32
n_batch = 512
n_threads=4
llm = LlamaCpp(
    model_path="/content/"+filename,
    n_threads=n_threads,
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    callback_manager=callback_manager,
    n_ctx=2048,
    temperature=0.0,
    repeat_penalty=1.18,
    top_p=1,
    top_k=3,
    max_tokens=256,
    streaming=True,
    #verbose=True,
)

In [ ]:
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="choosing native species is a good idea for start a restoration forest process?")